In [1]:
%cd ..
%env LOGLEVEL=ERROR

/home/tk/repos/explicit-memory
env: LOGLEVEL=ERROR


In [4]:
from memory import Memory, SemanticMemory, EpisodicMemory
from memory.environments import MemorySpace, OQAGenerator
import numpy as np

In [5]:
import random
from tqdm.notebook import tqdm

NUM_OUTER_LOOP = 100
NUM_INNER_LOOP = 1000
EPISOIC_MAX = 4
SEMANTIC_MAX = 4
HISTORY_MAX = 8


for space_type in tqdm(
    [
        "episodic_memory_manage",
        "episodic_question_answer",
        "semantic_memory_manage",
        "semantic_question_answer",
        "episodic_to_semantic",
        "episodic_semantic_question_answer",
    ]
):
    print(space_type)
    for _ in tqdm(range(NUM_OUTER_LOOP)):
        capacity = {
            "episodic": random.randint(1, EPISOIC_MAX),
            "semantic": random.randint(1, SEMANTIC_MAX),
        }
        if space_type in ["episodic_memory_manage", "episodic_question_answer"]:
            capacity["semantic"] = 0
        elif space_type in ["semantic_memory_manage", "semantic_question_answer"]:
            capacity["episodic"] = 0
        memspace = MemorySpace(
            capacity=capacity, space_type=space_type, max_history=HISTORY_MAX
        )

        for __ in range(NUM_INNER_LOOP):
            sample = memspace.sample()
            assert memspace.contains(
                sample
            ), f"{'space_type: ' + space_type:<40} {'capacity: ' + str(capacity):<40}"

            if space_type == "episodic_memory_manage":
                foo = memspace.numbers_to_episodic_memories(sample)
                bar = memspace.M_e.entries.copy()
                assert len(foo) == len(bar)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert abs(foo_[3] - bar_[3]) < 1

            elif space_type == "episodic_question_answer":
                foo = memspace.numbers_to_episodic_memories(sample[:-1, :])
                qa = memspace.numbers_to_episodic_question_answer(
                    sample[-1, :].reshape(1, -1)
                )
                bar = memspace.M_e.entries.copy()
                assert len(foo) == len(bar)
                assert memspace.is_correct_question_answer("episodic", qa)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert abs(foo_[3] - bar_[3]) < 1

            elif space_type == "semantic_memory_manage":
                foo = memspace.numbers_to_semantic_memories(sample)
                bar = memspace.M_s.entries.copy()
                assert len(foo) == len(bar)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert foo_[3] == bar_[3]

            elif space_type == "semantic_question_answer":
                foo = memspace.numbers_to_semantic_memories(sample[:-1, :])
                qa = memspace.numbers_to_semantic_question_answer(
                    sample[-1, :].reshape(1, -1)
                )
                bar = memspace.M_s.entries.copy()
                assert len(foo) == len(bar)
                assert memspace.is_correct_question_answer("semantic", qa)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert foo_[3] == bar_[3]

            elif space_type == "episodic_to_semantic":
                foo = memspace.numbers_to_episodic_memories(sample)
                bar = memspace.M_e.entries.copy()
                assert len(foo) == len(bar)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert abs(foo_[3] - bar_[3]) < 1

            elif space_type == "episodic_semantic_question_answer":
                qa = memspace.numbers_to_episodic_question_answer(
                    sample[-1, :].reshape(1, -1)
                )
                assert memspace.is_correct_question_answer("episodic", qa)

                episodic_memories = []
                semantic_memories = []
                for row in sample[:-1, :]:
                    if (
                        memspace.oqag.number2string(row[0])
                        == memspace.oqag.number2string(row[1])
                        == memspace.oqag.number2string(row[2])
                        == memspace.oqag.number2string(row[3])
                        == memspace.oqag.number2string(row[4])
                        == memspace.oqag.number2string(row[5])
                        == "<pad>"
                    ):
                        continue
                    else:
                        if (
                            memspace.oqag.number2string(row[0])
                            == memspace.oqag.number2string(row[3])
                            == "<pad>"
                        ):
                            semantic_memories.append(row)
                        else:
                            episodic_memories.append(row)

                episodic_memories = np.array(episodic_memories)
                semantic_memories = np.array(semantic_memories)

                foo = memspace.numbers_to_episodic_memories(episodic_memories)
                bar = memspace.M_e.entries.copy()
                assert len(foo) == len(bar)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert abs(foo_[3] - bar_[3]) < 1

                foo = memspace.numbers_to_semantic_memories(semantic_memories)
                bar = memspace.M_s.entries.copy()
                assert len(foo) == len(bar)

                for foo_, bar_ in zip(foo, bar):
                    assert foo_[0] == bar_[0]
                    assert foo_[1] == bar_[1]
                    assert foo_[2] == bar_[2]
                    assert foo_[3] == bar_[3]


  0%|          | 0/6 [00:00<?, ?it/s]

episodic_memory_manage


  0%|          | 0/100 [00:00<?, ?it/s]

AssertionError: 

In [12]:
memspace.M_e

{   '_frozen': False,
    'capacity': 3,
    'entries': [   [   "Jennifer's "
                       'clock',
                       'AtLocation',
                       "Jennifer's "
                       'desk',
                       158301.8235898018],
                   [   "James's "
                       'bench',
                       'AtLocation',
                       "James's "
                       'closet',
                       158301.82362484932],
                   [   "Michael's "
                       'bowl',
                       'AtLocation',
                       "Michael's "
                       'garage',
                       158301.8236477375]],
    'type': 'episodic'}

In [13]:
memspace.M_s

{   '_frozen': False,
    'capacity': 3,
    'entries': [   [   'sandwich',
                       'AtLocation',
                       'sky',
                       1]],
    'type': 'semantic'}